In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, numpy as np
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
%matplotlib widget

In [ ]:
from LinearMPC.MPCVelControl import MPCVelControl
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis, plot_static_states_inputs

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:
Ts = 0.05
sim_time = 15.0 
H = 5.0
x0 = np.array([0, 0, 0, 0, 0, 0, 5, 5, 10, 0, 0, 1])
x_target = np.zeros((12,))

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl.new_controller(rocket, Ts, H)

rocket.mass = 2.0
rocket.fuel_rate = 0.1

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, ref = rocket.simulate_control(
    mpc, sim_time, H, x0, x_target=x_target, method='nonlinear'
)

print(f"\nSimulation completed. Final time: {t_cl[-1]:.2f}s")
print(f"Final position: x={x_cl[9,-1]:.2f}, y={x_cl[10,-1]:.2f}, z={x_cl[11,-1]:.2f}")
print(f"Final velocity: vx={x_cl[6,-1]:.2f}, vy={x_cl[7,-1]:.2f}, vz={x_cl[8,-1]:.2f}")

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, Ref=ref[:, :-1], T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol);

In [ ]:
# Plot the trajectory
plot_static_states_inputs(t_cl[:-1], x_cl[:, :-1], u_cl, ref[:, :-1])
